<h1>Desarrollo del Desafio 1</h1>

<p>La inmobiliaria Properati publica periódicamente información sobre ofertas de propiedades para venta y alquiler. Ud. deberá asesorar a la inmobiliaria a desarrollar un modelo de regresión que permita predecir el precio por metro cuadrado de una propiedad. El objetivo             
final es que el modelo que desarrollen sea utilizado como tasador automático a ser aplicados a las próximas propiedades que sean comercializadas por la empresa. Para ello la empresa le provee de un dataset correspondiente al primer semestre de 2017.</p>

El dataset es de tamaño entre pequeño y mediano, pero tiene dos complejidades a las que deberá prestarle atención:
 
<ul>
    <li>Peso de missing data en algunas variables relevantes.</li>
    <li>Será importante tener en cuenta el problema de la influencia espacial en los precios por metro cuadrado. En efecto, es probable que existan diferencias importantes de en las diferentes geografías, barrios y zonas analizadas.</li>
</ul>

<h2>Objetivos:</h2>

<ul>
    <li>Efectuar una limpieza del dataset provisto. Particularmente, deberá diseñar estrategias para lidiar con los datos perdidos en ciertas variables.</li>
    <li>Realizar un análisis descriptivo de las principales variables.</li>
    <li>Crear nuevas columnas a partir de las características dadas que puedan tener valor predictivo.</li>
</ul>

<h2>Paso a paso:</h2>

1. AED Análisis exploratorio (para hacer/mirar este finde): ver qué tipo de datos tiene el dataset, qué datos numéricos/texto hay y cualquier cosa que se nos ocurra que podamos resaltar. La columna de descripción quizás tenga los amenities. Sacar cantidad de datos nulos y porcentaje
1. Limpieza: separar las columnas de lugar, verificar que las columnas de precio tengan los tipos correctos, columnas de lat-lon, sacar caracteres especiales de las columnas de texto, sacar signos de monedas, RE para columna de descripción
1. Verificar si hay datos duplicados (qué columnas tomamos en cuenta para esto, hay algún ID?)
1. Datos faltantes: tomar decisiones sobre qué recomponer y qué registros eliminar
1. a) Agrupar y sacar estadísticas: por barrio, por provincia, por m2, qué correlaciones hay entre las variables, qué variables nuevas se pueden sacar o agregar (cotización, por ejemplo)
  b) Es necesario crear variables dummies? (para rent/sell, para tipo de propiedad)
1.  Visualización: elegir gráficos que ilustren los datos 

<h3>Importar las librerias que estaremos utilizando.</h3>

In [1]:
import numpy as np
import pandas as pd
import chardet
import re

In [2]:
# Funcion especial para autodetectar el encoding. La eliminamos porque demora mucho!!!
# def get_encoding_type(csv_path):
#     rawdata = open(csv_path, 'rb').read()
#     result = chardet.detect(rawdata)
#     return result.get('encoding')

In [3]:
#Se define la ruta de la información.
data_propiedades = "../Data/properati.csv"

In [4]:
#Leemos los datos del archivo
data = pd.read_csv(data_propiedades, sep=",", encoding="UTF-8")

In [5]:
#Chequeamos que los datos se hayan importado correctamente
data.head(20)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...
3,3,sell,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15boh_venta_ph_lin...,PH 3 ambientes con patio. Hay 3 deptos en lote...,PH 3 amb. cfte. reciclado,https://thumbs4.properati.com/3/DgIfX-85Mog5SP...
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,35.0,1828.571429,1828.571429,NaN,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...
5,5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...
6,6,sell,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,...,78.0,1226.415094,1666.666667,NaN,NaN,NaN,http://www.properati.com.ar/15bor_venta_ph_mun...,MUY BUEN PH AL FRENTE CON ENTRADA INDEPENDIENT...,"MUY BUEN PH AL FRENTE DOS DORMITORIOS , PATIO,...",https://thumbs4.properati.com/5/6GOXsHCyDu1aGx...
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,40.0,3066.666667,3450.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,60.0,3000.000000,3250.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...
9,9,sell,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15box_venta_casa_r...,MEDNOZA AL 7600A UNA CUADRA DE CALLE MENDOZAWH...,WHITE 7637 - 2 DORMITORIOS CON PATIO,https://thumbs4.properati.com/8/RCf1YEWdF4rv98...


In [6]:
#Chequeamos cantidad de registros y cantidad de variables
data.shape

(121220, 26)

In [7]:
#Vemos qué tipos de datos tienen los campos
data.dtypes

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

**Cantidad de nulos por columna**

In [8]:
#Cantidad de nulos por columna
cant_nulos_por_campo = data.apply(lambda x: x.isnull().sum(), axis = 0)
print(cant_nulos_por_campo)

Unnamed: 0                         0
operation                          0
property_type                      0
place_name                        23
place_with_parent_names            0
country_name                       0
state_name                         0
geonames_id                    18717
lat-lon                        51550
lat                            51550
lon                            51550
price                          20410
currency                       20411
price_aprox_local_currency     20410
price_aprox_usd                20410
surface_total_in_m2            39328
surface_covered_in_m2          19907
price_usd_per_m2               52603
price_per_m2                   33562
floor                         113321
rooms                          73830
expenses                      106958
properati_url                      0
description                        2
title                              0
image_thumbnail                 3112
dtype: int64


**Porcentaje de nulos por columna**

In [9]:
cant_registros = data.shape[0]
#print(cant_registros)
#len(data.index) es lo mismo

porc_nulos_por_campo = np.round((100 * cant_nulos_por_campo / cant_registros),2)
print(porc_nulos_por_campo)

Unnamed: 0                     0.00
operation                      0.00
property_type                  0.00
place_name                     0.02
place_with_parent_names        0.00
country_name                   0.00
state_name                     0.00
geonames_id                   15.44
lat-lon                       42.53
lat                           42.53
lon                           42.53
price                         16.84
currency                      16.84
price_aprox_local_currency    16.84
price_aprox_usd               16.84
surface_total_in_m2           32.44
surface_covered_in_m2         16.42
price_usd_per_m2              43.39
price_per_m2                  27.69
floor                         93.48
rooms                         60.91
expenses                      88.23
properati_url                  0.00
description                    0.00
title                          0.00
image_thumbnail                2.57
dtype: float64


**Algunos datos que pueden servir para la limpieza**

In [10]:
property_type_count = data.property_type.value_counts()
property_type_count

apartment    71065
house        40268
PH            5751
store         4136
Name: property_type, dtype: int64

In [11]:
property_operation_count = data.operation.value_counts()
property_operation_count

sell    121220
Name: operation, dtype: int64

In [12]:
country_name_count = data.country_name.value_counts()
country_name_count

Argentina    121220
Name: country_name, dtype: int64

In [13]:
#Todos los valores de la columna country_name son Argentina 
#de la columna place_with_parents_name se puede reemplazar Argentina por string vacio

In [14]:
len(data.place_with_parent_names.index)

121220

In [15]:
# data.place_with_parent_names.sample(30)

In [16]:
patron_arg = '^\|Argentina\|'
#RE busca |Argentina al comienzo del string
patron_arg_regex = re.compile(patron_arg)

In [17]:
#creo una variable nueva para hacer el apply y reemplazo "|Argentina" por string vacio 
serie_place_with_parent_names = data.place_with_parent_names
place_parent_name_sin_arg = serie_place_with_parent_names.apply(lambda x: patron_arg_regex.sub("", x))

In [18]:
#chequeo los valores que quedaron
place_parent_name_sin_arg.head()

0                           Capital Federal|Mataderos|
1                     Bs.As. G.B.A. Zona Sur|La Plata|
2                           Capital Federal|Mataderos|
3                             Capital Federal|Liniers|
4    Buenos Aires Costa Atlántica|Mar del Plata|Cen...
Name: place_with_parent_names, dtype: object

In [19]:
patron_pipe_final = '\|$'
#RE busca | al final del string
regex_pipe_final = re.compile(patron_pipe_final)

In [20]:
#Reemplazo el pipe por string vacio así despues puedo splitear la cadena
place_parent_final = place_parent_name_sin_arg.apply(lambda x: regex_pipe_final.sub("", x))
place_parent_final.head()

0                            Capital Federal|Mataderos
1                      Bs.As. G.B.A. Zona Sur|La Plata
2                            Capital Federal|Mataderos
3                              Capital Federal|Liniers
4    Buenos Aires Costa Atlántica|Mar del Plata|Centro
Name: place_with_parent_names, dtype: object

<strong>Comentario:</strong> yo armaría tres campos a completar con estos datos, provincia, ciudad, municipio, donde municipio podria tener valores NaN, en el caso de capital le sumaría Bs.As. GBA / Bs.As. Interior, no se si eliminaria Bs.As. Costa Atlántica. Para pensar. 

**Segundo intento de separar columnas: con split (el primer intento fue con regex, demasiado complicado sin necesidad)**

In [21]:
serie_lista_place_split = place_parent_final.apply(lambda x: x.split("|"))
serie_lista_place_split.head(10)

0                         [Capital Federal, Mataderos]
1                   [Bs.As. G.B.A. Zona Sur, La Plata]
2                         [Capital Federal, Mataderos]
3                           [Capital Federal, Liniers]
4    [Buenos Aires Costa Atlántica, Mar del Plata, ...
5                           [Entre Ríos, Gualeguaychú]
6     [Bs.As. G.B.A. Zona Norte, Vicente López, Munro]
7                          [Capital Federal, Belgrano]
8                          [Capital Federal, Belgrano]
9                                  [Santa Fe, Rosario]
Name: place_with_parent_names, dtype: object

In [22]:
# Veo las cantidades de elementos en las listas que quedaron luego del split
# veo que hay registros con 4 elementos
serie_len = serie_lista_place_split.apply(len)
serie_len.value_counts()

2    76000
3    39892
1     4780
4      548
Name: place_with_parent_names, dtype: int64

In [23]:
#armo funciones para extraer el elemento que me interesa de cada lista según la posicion
#también considero la opcion de Capital Federal, donde la logica es distinta

def get_barrio(x):
    if x[0] == 'Capital Federal':
        barrio = x[2] if len(x) == 3 else np.NaN
    else:
        barrio = x[3] if (len(x) == 4) else np.NaN
    return barrio
        
def get_localidad(x):
    if x[0] == 'Capital Federal':
        localidad = x[1] if (len(x) == 3 or len(x) == 2) else np.NaN
    else:
        localidad = x[2] if (len(x) == 4) else (x[1] if (len(x) == 3 or len(x) == 2) else x[0])
    return localidad

def get_partido(x):
    if x[0] == 'Capital Federal':
        partido = np.NaN
    else: 
        partido = x[1] if (len(x) == 4 or len(x) == 3) else np.NaN
    return partido

def get_provincia(x):
    return np.NaN if len(x) == 1 else x[0]

In [24]:
#aplico la funcion para crear 4 series distintas
serie_barrio = serie_lista_place_split.apply(get_barrio)

serie_localidad = serie_lista_place_split.apply(get_localidad)

serie_partido = serie_lista_place_split.apply(get_partido)

serie_provincia = serie_lista_place_split.apply(get_provincia)


In [28]:
serie_barrio.sample(10)
# serie_localidad.sample(10)
# serie_partido.sample(10)
# serie_provincia.sample(10)

777       NaN
44826     NaN
48900     NaN
88623     NaN
115925    NaN
120061    NaN
58847     NaN
55773     NaN
67143     NaN
12686     NaN
Name: place_with_parent_names, dtype: object

In [26]:
# serie_localidad.head(15)

In [27]:
#armo un df nuevo para ver cómo quedó la separación
frame = { 'Provincia': serie_provincia, 'Partido': serie_partido,'Localidad': serie_localidad, 'Barrio': serie_barrio  } 
  
df_place_names = pd.DataFrame(frame) 
df_place_names.head(20)

,Provincia,Partido,Localidad,Barrio
0,Capital Federal,NaN,Mataderos,NaN
1,Bs.As. G.B.A. Zona Sur,NaN,La Plata,NaN
2,Capital Federal,NaN,Mataderos,NaN
3,Capital Federal,NaN,Liniers,NaN
4,Buenos Aires Costa Atlántica,Mar del Plata,Mar del Plata,NaN
5,Entre Ríos,NaN,Gualeguaychú,NaN
6,Bs.As. G.B.A. Zona Norte,Vicente López,Vicente López,NaN
7,Capital Federal,NaN,Belgrano,NaN
8,Capital Federal,NaN,Belgrano,NaN
9,Santa Fe,NaN,Rosario,NaN


**Comentarios**
1. La division en esas columnas puede variar, solo es cuestión de poner en las funciones que la última posición siempre quede en Barrio y no en Localidad como pensaba yo
1. En el dataset original existe el campo **Place** (que es siempre el último elemento del string de **placer_with_parent_name** y el campo **state** con lo cual creo que todo esto no era necesario, pero por lo menos me sirvió para practicar algunas cosas

In [38]:
description_series = data['description']
description_series.sample(15)

14613     Venta depto. 1 amb. Buen estado, cocina incorp...
105394    Departamento de 2 ambientes al frente del Pase...
65231     Te invitamos a conocer: En Nueva Córdoba, Mari...
79276     Departamento de 1 dormitorio con balcón cerrad...
79137     APTO CREDITO.  Impecable  departamento de 2 am...
36819     Venta de departamento de 4 ambientes con coche...
53978     *** VENTA CON RENTA HASTA AGOSTO 2017 ***Casa ...
106878    Venta de Casa 4 DORMITORIOS con  una muy buena...
116523    LA LUCILA. LIBERTADOR AL RÍO. ESPECTACULAR PIS...
35427     Venta de Departamento 1 AMBIENTE en Quilmes, Q...
71343     Departamento de dos ambientes, con cocina sepa...
117540    ESPECTACULAR Departamento de 2 ambientes en ed...
117032    CODIGO: 1092-EBE-7716 ubicado en: DEBENEDETI 7...
19777     Yerbal 6350, CABA /  PH 2 ambientes 50M2, mas ...
10799                            Galpon en Venta en Cordoba
Name: description, dtype: object

In [15]:
amb1_pattern = "(?P<numero_amb>\d(\s)?(amb|AMB))"
amb1_pattern_regex = re.compile(amb1_pattern)



In [20]:
amb1_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else\
                                      amb1_pattern_regex.search(x))
mask_amb1_match_notnull = amb1_match.notnull()

data.loc[mask_amb1_match_notnull, 'rooms_clean'] = \
amb1_match[mask_amb1_match_notnull].apply(lambda x: x.group("numero_amb"))


In [21]:
data['rooms_clean'].head(15)

0     2 AMB
1       NaN
2     2 AMB
3     3 amb
4       NaN
5       NaN
6       NaN
7       NaN
8       NaN
9       NaN
10      NaN
11    1 amb
12    2 AMB
13    3 amb
14      NaN
Name: rooms_clean, dtype: object

In [18]:
data['rooms_clean'].notnull().sum()

35650

In [29]:
dorm1_pattern = "(?P<numero_dorm>\d(\s)?(dorm|DORM))"
dorm1_pattern_regex = re.compile(dorm1_pattern)

In [30]:
dorm1_match = description_series.apply(lambda x: x if (x is np.NaN) | (x is None) else dorm1_pattern_regex.search(x))

mask_dorm1_match_notnull = dorm1_match.notnull()


In [31]:
data.loc[mask_dorm1_match_notnull, 'rooms_clean'] = dorm1_match[mask_dorm1_match_notnull].apply(lambda x: x.group("numero_dorm"))

In [35]:
data['rooms_clean'].head(15)
data['rooms_clean'].notnull().sum()

64063

In [39]:
#Falta Ambiente unico, mono(\s)?ambiente, 1 y 1/2 amb

In [ ]:
#PISOS: \d piso, planta baja, PH = planta baja, \der\spiso, con palabras?